In [2]:
import pygambit as gbt

In [3]:
# define shot types
options  = [["easy", gbt.Rational(2,3), gbt.Rational(2,3)],
            ["hard", gbt.Rational(1,3), gbt.Rational(1,3)]]
names = [] # this an array of just the names for when a player choses a shot
for i in options: names.append(i[0])
names

['easy', 'hard']

Here is a game with just one round

In [4]:
# g is one round of horse
r = gbt.Game.new_tree(players=["player1","player2"], title="round of horse")
# set outcomes
pl_one_letter = r.add_outcome([-1,0], label = "player 1 letter")
pl_two_letter = r.add_outcome([0,-1], label = "player 2 letter")
no_letter = r.add_outcome([0,0], label = "no letter given") # this one isn't used

r.append_move(r.root, "player1", names) #p1 choses a shot
for i in range(len(options)):
    r.append_move(r.root.children[i], r.players.chance, ["hit", "miss"])
    r.set_chance_probs(r.root.children[i].infoset, [options[i][1], (1 - options[i][1])]) #set the chance that p1 hits
    # if p1 hits:
    r.append_move(r.root.children[i].children[0], r.players.chance, ["hit", "miss"])
    r.set_chance_probs(r.root.children[i].children[0].infoset, [options[i][2], (1 - options[i][2])])
    r.set_outcome(r.root.children[i].children[0].children[0], pl_one_letter)
    r.set_outcome(r.root.children[i].children[0].children[1], pl_two_letter)
    # if p1 misses:
    r.set_outcome(r.root.children[i].children[1], pl_one_letter)
r

Game(title='round of horse')

In [5]:
result = gbt.nash.lcp_solve(r)
eqm = result.equilibria[0]
eqm

[[[Rational(0, 1), Rational(1, 1)]], []]

Here is a full game:

In [6]:
# h is a full game of horse
# WARNING
# be careful running this it can take a while at high round numbers
# and trying to do anything with it will likely crash jupyter
rounds = 2
h = gbt.Game.new_tree(players=["player1","player2"], title="full game of horse")
# set outcomes
pl_one_win = h.add_outcome([1,0], label = "player 1 wins")
pl_two_win = h.add_outcome([0,1], label = "player 2 wins")

def populate(g, node, playerup, playerdown, letters_one, letters_two):
    g.append_move(node, playerup, names) #playerup choses a shot
    if letters_one == rounds:
        g.set_outcome(node, pl_two_win)
    elif letters_two == rounds:
        g.set_outcome(node, pl_one_win)
    else:
        for i in range(len(options)): # i here represents the diffrent shot types
            g.append_move(node.children[i], g.players.chance, ["hit", "miss"])
            g.set_chance_probs(node.children[i].infoset, [options[i][1], (1 - options[i][1])]) #set the chance that playerup hits
            # if playerup hits:
            g.append_move(node.children[i].children[0], g.players.chance, ["hit", "miss"])
            g.set_chance_probs(node.children[i].children[0].infoset, [options[i][2], (1 - options[i][2])]) #chance that playerdown hits
            populate(g, node.children[i].children[0].children[1], playerdown, playerup, (letters_one + 1), letters_two)
            populate(g, node.children[i].children[0].children[0], playerup, playerdown, (letters_one), letters_two + 1)
            # if playerup misses:
            populate(g, node.children[i].children[1], playerup, playerdown, (letters_one + 1), letters_two)
populate(h, h.root, 'player1', 'player2', 0, 0)
print('finished')

finished


In [ ]:
# WARNING
# do not run this
# it will crash jupyter
result = gbt.nash.lcp_solve(h)
eqm = result.equilibria[0]
result
print('done')